In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

매뉴얼 URL 리스트

In [14]:
urls = [
    "https://downloadcenter.samsung.com/content/PM/202501/20250103095851140/KO/OID84030_RF9000D_2023_UM_DA68-04789A_KO_Ver34_KO_c6_s1.html",
    "https://downloadcenter.samsung.com/content/PM/202501/20250103095851140/KO/OID84030_RF9000D_2023_UM_DA68-04789A_KO_Ver34_KO_c6_s2.html",
    "https://downloadcenter.samsung.com/content/PM/202501/20250103095851140/KO/OID84030_RF9000D_2023_UM_DA68-04789A_KO_Ver34_KO_c6_s3.html",
    "https://downloadcenter.samsung.com/content/PM/202501/20250103095851140/KO/OID84030_RF9000D_2023_UM_DA68-04789A_KO_Ver34_KO_c6_s4.html",
    "https://downloadcenter.samsung.com/content/PM/202501/20250103095851140/KO/OID84030_RF9000D_2023_UM_DA68-04789A_KO_Ver34_KO_c6_s5.html",
    "https://downloadcenter.samsung.com/content/PM/202501/20250103095851140/KO/OID84030_RF9000D_2023_UM_DA68-04789A_KO_Ver34_KO_c6_s6.html",
    "https://downloadcenter.samsung.com/content/PM/202501/20250103095851140/KO/OID84030_RF9000D_2023_UM_DA68-04789A_KO_Ver34_KO_c6_s7.html",
    "https://downloadcenter.samsung.com/content/PM/202501/20250103095851140/KO/OID84030_RF9000D_2023_UM_DA68-04789A_KO_Ver34_KO_c6_s8.html",
    "https://downloadcenter.samsung.com/content/PM/202501/20250103095851140/KO/OID84030_RF9000D_2023_UM_DA68-04789A_KO_Ver34_KO_c6_s9.html"
]

사용자 매뉴얼 크롤링

In [15]:
# 결과를 저장할 리스트
data = []

# URL 요청 및 HTML 파싱
for url in urls:
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, "html.parser")

    # main 영역 내부의 section 내에 있는 하위 section들 중 h3 태그가 있는 부분 찾기
    # (여기서는 main > section > section 선택자를 사용합니다)
    symptom_sections = soup.select("main section section")

    for section in symptom_sections:
        h3 = section.find("h3")
        if h3:
            # 고장/비고장 증상 텍스트 추출
            symptom = h3.get_text(strip=True)
            # 해당 section 내 ul 태그의 li 항목(해결방안) 추출
            ul = section.find("ul")
            if ul:
                # 여러 해결방안 항목이 있다면 줄바꿈으로 구분하여 하나의 문자열로 결합
                resolutions = "\n".join(li.get_text(strip=True) for li in ul.find_all("li"))
            else:
                resolutions = ""

            data.append({"증상": symptom, "해결방안": resolutions})

In [16]:
print(data)

[{'증상': '물 흐르는 소리가 나요! (꾸르륵, 지글지글, 벅벅 등)', '해결방안': "제품 내부의 냉매(차갑게 하는 물질)가 액체와 기체로 바뀔 때 나는 소리입니다.\n냉각기에 붙어있는 성에가 녹아 물이 되어 흘러내리면서 '칙칙', '벅벅' 등과 같은 소리가 납니다."}, {'증상': "냉장고 안에서 '뚝'하는 소리가 나요! (뚝, 딱, 뚜둑, 달그락 등)", '해결방안': '제품 안의 온도가 변할 때 부품이 늘어나거나 줄어들면서 나는 소리입니다.\n냉각기에 붙어있는 성에가 녹을 때에도 유사한 소리가 발생합니다.'}, {'증상': "'웅~', '웽~', '드르륵', '덜컥'하는 소리가 나요!", '해결방안': '제품의 작동이 시작되거나 끝날 때 압축기나 팬 등에 의해 발생하는 소리이며, 자동차의 시동을 켜거나 끌 때, 또는 가속할 때 발생하는 소리와 유사한\r\n                              원리입니다.'}, {'증상': '방금 설치했는데 소리가 크게 들려요!', '해결방안': '처음 제품을 작동하거나 정전 등으로 제품이 꺼졌다 켜졌을 때 제품 안의 온도를 빨리 낮추기 위해 압축기나 팬이 빠르게 회전하게 됩니다. 이때 소리가\r\n                              크게 들릴 수 있으며, 길게는 2 ~ 3일 이후 일정 온도 이하로 내려가면 압축기나 팬의 회전 속도가 낮아지며 소리가 줄어듭니다.'}, {'증상': '냉장고 문을 닫은 직후에 소리가 나요!', '해결방안': "문을 열었다 닫으면 제품 안으로 들어온 공기가 빠르게 냉각되면서, 내부 압력이 일시적으로 낮아져 제품 뒷면에서 '쉭~'하는 바람 새는 소리가 나거나,\r\n                              제품 내부 구조가 일시적으로 변형되면서 '뚜둑'과 같은 소리가 날 수 있습니다."}, {'증상': '소음이 심하고 이상한 소리가 나요.', '해결방안': '설치 장소의 바닥이 평평하면서 단단한 곳에 설치하세요.\n제품 뒷면과 옆면을 벽면에서

In [17]:
# 데이터프레임 생성: 필요 시 '순번' 컬럼 추가
df = pd.DataFrame(data)
df.insert(0, "순번", range(1, len(df) + 1))

In [18]:
# 결과 엑셀 파일로 저장
output_filename = "해결방안_리스트.xlsx"
df.to_excel(output_filename, index=False)
print(f"엑셀 파일 '{output_filename}'이(가) 생성되었습니다.")

엑셀 파일 '해결방안_리스트.xlsx'이(가) 생성되었습니다.
